In [ ]:
#hide
#default_exp cli
from nbdev.showdoc import show_doc

# Command line functions

> Console commands added by the nbdev library

In [ ]:
#export
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.merge import *
from nbdev.export2html import *
from nbdev.clean import *
from nbdev.test import *
from fastcore.script import *

`nbdev` comes with the following commands. To use any of them, you must be in one of the subfolders of your project: they will search for the `settings.ini` recursively in the parent directory but need to access it to be able to work. Their names all begin with nbdev so you can easily get a list with tab completion.
- `nbdev_build_docs` builds the documentation from the notebooks
- `nbdev_build_lib` builds the library from the notebooks
- `nbdev_bump_version` increments version in `settings.py` by one
- `nbdev_clean_nbs` removes all superfluous metadata form the notebooks, to avoid merge conflicts
- `nbdev_detach` exports cell attachments to `dest` and updates references
- `nbdev_diff_nbs` gives you the diff between the notebooks and the exported library
- `nbdev_fix_merge` will fix merge conflicts in a notebook file
- `nbdev_install_git_hooks` installs the git hooks that use the last two command automatically on each commit/merge
- `nbdev_nb2md` converts a notebook to a markdown file
- `nbdev_new` creates a new nbdev project
- `nbdev_read_nbs` reads all notebooks to make sure none are broken
- `nbdev_test_nbs` runs tests in notebooks
- `nbdev_trust_nbs` trusts all notebooks (so that the HTML content is shown)
- `nbdev_update_lib` propagates any change in the library back to the notebooks

## Navigating from notebooks to script and back

In [ ]:
show_doc(nbdev_build_lib)

<h4 id="nbdev_build_lib" class="doc_header"><code>nbdev_build_lib</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/export2html.py#L474" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_build_lib</code>(**`fname`**:"A notebook name or glob to convert"=*`None`*)

Export notebooks matching `fname` to python modules

By default (`fname` left to `None`), the whole library is built from the notebooks in the `lib_folder` set in your `settings.ini`.

In [ ]:
show_doc(nbdev_update_lib)

<h4 id="nbdev_update_lib" class="doc_header"><code>nbdev_update_lib</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/sync.py#L121" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_update_lib</code>(**`fname`**:"A notebook name or glob to convert"=*`None`*, **`silent`**:"Don't print results"=*`False`*)

Propagates any change in the modules matching `fname` to the notebooks that created them

By default (`fname` left to `None`), the whole library is treated. Note that this tool is only designed for small changes such as typo or small bug fixes. You can't add new cells in notebook from the library.

In [ ]:
show_doc(nbdev_diff_nbs)

<h4 id="nbdev_diff_nbs" class="doc_header"><code>nbdev_diff_nbs</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/sync.py#L139" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_diff_nbs</code>()

Prints the diff between an export of the library in notebooks and the actual modules

## Running tests

In [ ]:
show_doc(nbdev_test_nbs)

<h4 id="nbdev_test_nbs" class="doc_header"><code>nbdev_test_nbs</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/test.py#L100" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_test_nbs</code>(**`fname`**:"A notebook name or glob to convert"=*`None`*, **`flags`**:"Space separated list of flags"=*`None`*, **`n_workers`**:"Number of workers to use"=*`None`*, **`verbose`**:"Print errors along the way"=*`True`*, **`timing`**:"Timing each notebook to see the ones are slow"=*`False`*, **`pause`**:"Pause time (in secs) between notebooks to avoid race conditions"=*`0.5`*)

Test in parallel the notebooks matching `fname`, passing along `flags`

By default (`fname` left to `None`), the whole library is tested from the notebooks in the `lib_folder` set in your `settings.ini`.

## Building documentation

In [ ]:
show_doc(nbdev_build_docs)

<h4 id="nbdev_build_docs" class="doc_header"><code>nbdev_build_docs</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/export2html.py#L652" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_build_docs</code>(**`fname`**:"A notebook name or glob to convert"=*`None`*, **`force_all`**:"Rebuild even notebooks that haven't changed"=*`False`*, **`mk_readme`**:"Also convert the index notebook to README"=*`True`*, **`n_workers`**:"Number of workers to use"=*`None`*, **`pause`**:"Pause time (in secs) between notebooks to avoid race conditions"=*`0.5`*)

Build the documentation by converting notebooks matching `fname` to html

By default (`fname` left to `None`), the whole documentation is build from the notebooks in the `lib_folder` set in your `settings.ini`, only converting the ones that have been modified since the their corresponding html was last touched unless you pass `force_all=True`. The index is also converted to make the README file, unless you pass along `mk_readme=False`.

In [ ]:
show_doc(nbdev_nb2md)

<h4 id="nbdev_nb2md" class="doc_header"><code>nbdev_nb2md</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/export2html.py#L664" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_nb2md</code>(**`fname`**:"A notebook file name to convert", **`dest`**:"The destination folder"=*`'.'`*, **`img_path`**:"Folder to export images to"=*`''`*, **`jekyll`**:"To use jekyll metadata for your markdown file or not"=*`False`*)

Convert the notebook in `fname` to a markdown file

In [ ]:
show_doc(nbdev_detach)

<h4 id="nbdev_detach" class="doc_header"><code>nbdev_detach</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/export2html.py#L616" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_detach</code>(**`path_nb`**:"Path to notebook", **`dest`**:"Destination folder"=*`''`*, **`use_img`**:"Convert markdown images to img tags"=*`False`*, **`replace`**:"Write replacement notebook back to `path_bn`"=*`True`*)

Export cell attachments to `dest` and update references

## Other utils

In [ ]:
show_doc(nbdev_read_nbs)

<h4 id="nbdev_read_nbs" class="doc_header"><code>nbdev_read_nbs</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/test.py#L127" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_read_nbs</code>(**`fname`**:"A notebook name or glob to convert"=*`None`*)

Check all notebooks matching `fname` can be opened

By default (`fname` left to `None`), the all the notebooks in `lib_folder` are checked.

In [ ]:
show_doc(nbdev_trust_nbs)

<h4 id="nbdev_trust_nbs" class="doc_header"><code>nbdev_trust_nbs</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/sync.py#L155" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_trust_nbs</code>(**`fname`**:"A notebook name or glob to convert"=*`None`*, **`force_all`**:"Trust even notebooks that haven't changed"=*`False`*)

Trust notebooks matching `fname`

By default (`fname` left to `None`), the all the notebooks in `lib_folder` are trusted. To speed things up, only the ones touched since the last time this command was run are trusted unless you pass along `force_all=True`.

In [ ]:
show_doc(nbdev_fix_merge)

<h4 id="nbdev_fix_merge" class="doc_header"><code>nbdev_fix_merge</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/merge.py#L92" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_fix_merge</code>(**`fname`**:"A notebook filename to fix", **`fast`**:"Fast fix: automatically fix the merge conflicts in outputs or metadata"=*`True`*, **`trust_us`**:"Use local outputs/metadata when fast merging"=*`True`*)

Fix merge conflicts in notebook `fname`

When you have merge conflicts after a `git pull`, the notebook file will be broken and won't open in jupyter notebook anymore. This command fixes this by changing the notebook to a proper json file again and add markdown cells to signal the conflict, you just have to open that notebook again and look for `>>>>>>>` to see those conflicts and manually fix them. The old broken file is copied with a `.ipynb.bak` extension, so is still accessible in case the merge wasn't successful.

Moreover, if `fast=True`, conflicts in outputs and metadata will automatically be fixed by using the local version if `trust_us=True`, the remote one if `trust_us=False`. With this option, it's very likely you won't have anything to do, unless there is a real conflict.

In [ ]:
#export
def bump_version(version, part=2):
    version = version.split('.')
    version[part] = str(int(version[part]) + 1)
    for i in range(part+1, 3): version[i] = '0'
    return '.'.join(version)

In [ ]:
test_eq(bump_version('0.1.1'   ), '0.1.2')
test_eq(bump_version('0.1.1', 1), '0.2.0')

In [ ]:
#export
@call_parse
def nbdev_bump_version(part:Param("Part of version to bump", int)=2):
    "Increment version in `settings.py` by one"
    cfg = Config()
    print(f'Old version: {cfg.version}')
    cfg.d['version'] = bump_version(Config().version, part)
    cfg.save()
    update_version()
    print(f'New version: {cfg.version}')

## Git hooks

In [ ]:
#export
@call_parse
def nbdev_install_git_hooks():
    "Install git hooks to clean/trust notebooks automatically"
    try: path = Config().config_file.parent
    except: path = Path.cwd()
    hook_path = path/'.git'/'hooks'
    fn = hook_path/'post-merge'
    hook_path.mkdir(parents=True, exist_ok=True)
    #Trust notebooks after merge
    fn.write_text("#!/bin/bash\necho 'Trusting notebooks'\nnbdev_trust_nbs")
    os.chmod(fn, os.stat(fn).st_mode | stat.S_IEXEC)
    #Clean notebooks on commit/diff
    (path/'.gitconfig').write_text("""# Generated by nbdev_install_git_hooks
#
# If you need to disable this instrumentation do:
#   git config --local --unset include.path
#
# To restore the filter
#   git config --local include.path .gitconfig
#
# If you see notebooks not stripped, checked the filters are applied in .gitattributes
#
[filter "clean-nbs"]
        clean = nbdev_clean_nbs --read_input_stream True
        smudge = cat
        required = true
[diff "ipynb"]
        textconv = nbdev_clean_nbs --disp True --fname
""")
    cmd = "git config --local include.path ../.gitconfig"
    print(f"Executing: {cmd}")
    run(cmd)
    print("Success: hooks are installed and repo's .gitconfig is now trusted")
    try: nb_path = Config().path("nbs_path")
    except: nb_path = Path.cwd()
    (nb_path/'.gitattributes').write_text("**/*.ipynb filter=clean-nbs\n**/*.ipynb diff=ipynb\n")

This command installs git hooks to make sure notebooks are cleaned before you commit them to GitHub and automatically trusted at each merge. To be more specific, this creates:
- an executable '.git/hooks/post-merge' file that contains the command `nbdev_trust_nbs`
- a `.gitconfig` file that uses `nbev_clean_nbs` has a filter/diff on all notebook files inside `nbs_folder` and a `.gitattributes` file generated in this folder (copy this file in other folders where you might have notebooks you want cleaned as well)

## Starting a new project

In [ ]:
#export
_template_git_repo = "https://github.com/fastai/nbdev_template.git"

In [ ]:
#export
import tarfile

In [ ]:
#export
def extract_tgz(url, dest='.'):
    with urlopen(url) as u: tarfile.open(mode='r:gz', fileobj=u).extractall(dest)

In [ ]:
#export
@call_parse
def nbdev_new():
    "Create a new nbdev project from the current git repo"
    url = run('git config --get remote.origin.url')
    if not url: raise Exception('This does not appear to be a cloned git directory with a remote')
    author = run('git config --get user.name').strip()
    email = run('git config --get user.email').strip()
    if not (author and email): raise Exception('User name and email not configured in git')

    # download and untar template, and optionally notebooks
    FILES_URL = 'https://files.fast.ai/files/'
    extract_tgz(f'{FILES_URL}nbdev_files.tgz')
    path = Path()
    for o in (path/'nbdev_files').ls(): 
        if not Path(f'./{o.name}').exists(): shutil.move(str(o), './') 
    shutil.rmtree('nbdev_files')
    if first(path.glob('*.ipynb')): print("00_core.ipynb not downloaded since a notebook already exists.")
    else: urlsave(f'{FILES_URL}00_core.ipynb')
    if not (path/'index.ipynb').exists(): urlsave(f'{FILES_URL}index.ipynb')

    # auto-config settings.ini from git
    settings_path = Path('settings.ini')
    settings = settings_path.read_text()
    owner,repo = repo_details(url)
    branch = run('git symbolic-ref refs/remotes/origin/HEAD').strip().split('/')[-1]
    settings = settings.format(lib_name=repo, user=owner, author=author, author_email=email, branch=branch)
    settings_path.write_text(settings)
    
    nbdev_install_git_hooks()
    if not (path/'LICENSE').exists() and not (path/'LICENSE.md').exists():
        warnings.warn('No LICENSE file found - you will need one if you will create pypi or conda packages.')

`nbdev_new` is a command line tool that creates a new nbdev project from the current directory, which must be a cloned git repo.

After you run `nbdev_new`, please check the contents of `settings.ini` look good, and then run `nbdev_build_lib`.

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 99_search.ipynb.
Converted example.ipynb.
Converted index.ipynb.
Converted try_save.ipynb.
Converted tutorial.ipynb.
